## Importing core libraries and custom estimators

In [1]:
# Importing core libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score, precision_score, confusion_matrix
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope

# Importing custom estimators and evaluation tool
from CustomEstimator import MultivariateGaussian, MultivariateTDistribution, SimpleAnomalyDetector

## Loading data - Credit Card Fraud dataset
#### Context on the dataset

V1 to V28 are PCA components of the data. Only features 'Time' and 'Amount' are untransformed features.

For more details about the dataset, visit the [Kaggle site here](https://www.kaggle.com/mlg-ulb/creditcardfraud).

>The datasets contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. 
The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.
It contains only numerical input variables which are the result of a PCA transformation.

In [2]:
# Importing the Credit Card Fraud dataset
data = pd.read_csv('creditcard.csv')
y_data = data.copy()['Class'].values
original_data = data.copy()

# Clean data set
normal_only_data = data[data['Class']==0]
print('Normal only data shape: ', normal_only_data.shape)
# Fraud data set
fraud_only_data = data[data['Class']==1]
print('Fraud only data shape: ', fraud_only_data.shape)

# Shuffling the data
normal_only_data = normal_only_data.sample(frac=1, random_state=42)
fraud_only_data = fraud_only_data.sample(frac=1, random_state=42)

# 80/10/10 data split for normal data
train_set, dev_set, test_set = np.split(normal_only_data, [int(0.8*len(normal_only_data)), int(0.9*len(normal_only_data))])
train_set = train_set.drop('Class', axis=1)

# 50/50 data split for fraud data
fraud_set_1, fraud_set_2 = np.split(fraud_only_data, [int(0.5*len(fraud_only_data))])

# Appending fraud data to dev and test set
dev_set = dev_set.append(fraud_set_1)
y_dev_set = dev_set['Class']
dev_set = dev_set.drop('Class', axis=1)
test_set = test_set.append(fraud_set_2)
y_test_set = test_set['Class']
test_set = test_set.drop('Class', axis=1)

# Showing shapes
for name, data in zip(['Train data shape: ', 'Dev data shape: ', 'Test data shape: '],[train_set, dev_set, test_set]):
    print(name, data.shape)

Normal only data shape:  (284315, 31)
Fraud only data shape:  (492, 31)
Train data shape:  (227452, 30)
Dev data shape:  (28677, 30)
Test data shape:  (28678, 30)


In [3]:
# Showing the first few rows of the data
original_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## Training and testing models - Novelty detection
For simplicity, I will train and test the custom and a few scikit-learn models on a novelty detection bias.

Novelty detection uses clean data (normal) only to train, and predict on contaminated data. Whereas Outlier detection uses contaminated (normal + fraud) data to train.

For more information, refer to scikit-learn's [Guidance](https://scikit-learn.org/stable/modules/outlier_detection.html#outlier-detection) here

In [3]:
# Helper function to evaluate models
labels = ['Normal', 'Fraud']
def evaluate_model(y_true, y_preds, labels=labels):
    cm = confusion_matrix(y_true, y_preds)
    print('Recall score:\n', recall_score(y_true, y_preds))
    print('Precision score:\n', precision_score(y_true, y_preds))
    print('Confusion matrix:\n')
    cm_df = pd.DataFrame({'Normal (predicted)': (cm[0, 0], cm[1, 0]),
                         'Fraud (predicted)': (cm[0, 1], cm[1, 1])},
                        index=['Normal (true)', 'Fraud (true)'])
    print(cm_df)

In [5]:
# Training Multivariate Gaussian Anomaly Detector
mvg = MultivariateGaussian(epsilon=0.05**30)
mvg.fit(train_set)

MultivariateGaussian(epsilon=9.3132257461548e-40)

In [6]:
# Evaluating on the Dev set
mvg_y_dev_preds = mvg.predict(dev_set)
evaluate_model(y_dev_set, mvg_y_dev_preds)

Recall score:
 0.3780487804878049
Precision score:
 0.1706422018348624
Confusion matrix:

               Normal (predicted)  Fraud (predicted)
Normal (true)               27979                452
Fraud (true)                  153                 93


In [13]:
# Training Simple Anomaly Detector (SAD)
# Side note: I'm still thinking about a new name for this Estimator ... suggestions are welcomed
simple = SimpleAnomalyDetector(epsilon=0.05**30)
simple.fit(train_set)

SimpleAnomalyDetector(epsilon=9.3132257461548e-40)

In [14]:
# Evaluating on the Dev Set
y_simple_dev_preds = simple.predict(dev_set)
evaluate_model(y_dev_set, y_simple_dev_preds)

Recall score:
 0.8414634146341463
Precision score:
 0.22162740899357602
Confusion matrix:

               Normal (predicted)  Fraud (predicted)
Normal (true)               27704                727
Fraud (true)                   39                207


In [7]:
# Training Multivariate T Anomaly Detector
mvt = MultivariateTDistribution(epsilon=0.05**30, df=3)
mvt.fit(train_set)

MultivariateTDistribution(df=3, epsilon=9.3132257461548e-40)

In [8]:
# Evaluating on the Dev Set
mvt_y_dev_preds = mvt.predict(dev_set)
evaluate_model(y_dev_set, mvt_y_dev_preds)

Recall score:
 0.8130081300813008
Precision score:
 0.5012531328320802
Confusion matrix:

               Normal (predicted)  Fraud (predicted)
Normal (true)               28232                199
Fraud (true)                   46                200


In [10]:
# Training Local Outlier Factor Anomaly Detector
lof = LocalOutlierFactor(novelty=True, metric='euclidean')
lof.fit(train_set)

C:\Users\ZH834BT\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\neighbors\lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


LocalOutlierFactor(algorithm='auto', contamination='legacy', leaf_size=30,
                   metric='euclidean', metric_params=None, n_jobs=None,
                   n_neighbors=20, novelty=True, p=2)

In [11]:
# Evaluating on the Dev set
lof_y_dev_preds = lof.predict(dev_set)
lof_y_dev_preds[lof_y_dev_preds==1] = 0
lof_y_dev_preds[lof_y_dev_preds==-1] = 1
evaluate_model(y_dev_set, lof_y_dev_preds)

Recall score:
 0.4715447154471545
Precision score:
 0.03488721804511278
Confusion matrix:

               Normal (predicted)  Fraud (predicted)
Normal (true)               25222               3209
Fraud (true)                  130                116


In [12]:
# Training Isolation Forest Anomaly Detector
ifr = IsolationForest(random_state=42, behaviour="new")
ifr.fit(train_set)

C:\Users\ZH834BT\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


IsolationForest(behaviour='new', bootstrap=False, contamination='legacy',
                max_features=1.0, max_samples='auto', n_estimators=100,
                n_jobs=None, random_state=42, verbose=0, warm_start=False)

In [13]:
# Evaluating on the Dev Set
ifr_y_dev_preds = ifr.predict(dev_set)
ifr_y_dev_preds[ifr_y_dev_preds==1] = 0
ifr_y_dev_preds[ifr_y_dev_preds==-1] = 1
evaluate_model(y_dev_set, ifr_y_dev_preds)

Recall score:
 0.8861788617886179
Precision score:
 0.06982703395259449
Confusion matrix:

               Normal (predicted)  Fraud (predicted)
Normal (true)               25527               2904
Fraud (true)                   28                218


In [14]:
# Training Kmeans Clustering
kmeans = KMeans(n_clusters=2, algorithm='auto')
kmeans.fit(train_set)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [15]:
# Evaluating KMeans on the Dev Set
kmeans_y_dev_preds = kmeans.predict(dev_set)
# Since KMeans only does clustering, we can decide which cluster would be Normal and which cluster would be Fraud
kmeans_y_dev_preds = np.where(kmeans_y_dev_preds==1, 0, 1)
evaluate_model(y_dev_set, kmeans_y_dev_preds)

Recall score:
 0.34552845528455284
Precision score:
 0.006477671086724585
Confusion matrix:

               Normal (predicted)  Fraud (predicted)
Normal (true)               15394              13037
Fraud (true)                  161                 85


#### Contaminated dataset

In [16]:
# Trying out Isolation Forest on contaminated dataset
ifr_2 = IsolationForest(random_state=42, behaviour="new")
ifr_2_y_preds = ifr_2.fit_predict(original_data.drop('Class', axis=1))
ifr_2_y_preds[ifr_2_y_preds==1] = 0
ifr_2_y_preds[ifr_2_y_preds==-1] = 1
evaluate_model(y_data, ifr_2_y_preds)

C:\Users\ZH834BT\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


Recall score:
 0.8800813008130082
Precision score:
 0.015203117868052386
Confusion matrix:

               Normal (predicted)  Fraud (predicted)
Normal (true)              256267              28048
Fraud (true)                   59                433


## Trying out Neural Network Autoencoders

#### Intuition
The basic idea behind NN Autoencoders is to learn the very low-level representations of the data. After this process hopefully the 'noise' have been minimised from the data, and the result representations (outputs of NN Autoencoders) can be used as inputs to simplier classifiers such as Logistic Regression.

In [64]:
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# Considered using XGBoost, my it would take a while on my EY laptop

In [5]:
# I use smaller sample to save computational time
train_data = normal_only_data[:100000].append(fraud_only_data[:390])
train_data = train_data.sample(frac=1, random_state=42)
print('NN Train data shape:\n', train_data.shape)
dev_data = normal_only_data[100000:120000].append(fraud_only_data[390:])
dev_data = dev_data.sample(frac=1, random_state=42)
print('NN Dev data shape:\n', dev_data.shape)

NN Train data shape:
 (100390, 31)
NN Dev data shape:
 (20102, 31)


In [6]:
# Separating X and y
X_train_nn, y_train_nn = train_data.drop('Class', axis=1, inplace=False), train_data['Class'].values
scaler = MinMaxScaler()
X_train_nn = scaler.fit_transform(X_train_nn)
print(X_train_nn.shape, y_train_nn.shape)
X_dev_nn, y_dev_nn = dev_data.drop('Class', axis=1, inplace=False), dev_data['Class'].values
X_dev_nn = scaler.transform(X_dev_nn)
print(X_dev_nn.shape, y_dev_nn.shape)

(100390, 30) (100390,)
(20102, 30) (20102,)


In [53]:
# Building computational graph for NN Autocencoder
# Input layer
input_layer = Input(shape=(X_train_nn.shape[1],))

# Encoding part
encoded_1 = Dense(200, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoded_2 = Dense(100, activation='relu')(encoded_1)

# Decoding part
decoded_1 = Dense(100, activation='tanh')(encoded_2)
decoded_2 = Dense(200, activation='tanh')(decoded_1)

# Output layer
output_layer = Dense(X_train_nn.shape[1], activation='relu')(decoded_2)

# Compiling model
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer='adam', loss='mse')

In [54]:
# Training the model
autoencoder.fit(X_train_nn, X_train_nn,
                batch_size=256, epochs=20,
                shuffle=True, validation_split=0.2
               )

Epoch 1/20
314/314 [==============================] - 1s 3ms/step - loss: 0.0474 - val_loss: 0.0066
Epoch 2/20
314/314 [==============================] - 1s 2ms/step - loss: 0.0039 - val_loss: 6.5883e-04
Epoch 3/20
314/314 [==============================] - 1s 2ms/step - loss: 6.0444e-04 - val_loss: 5.1009e-04
Epoch 4/20
314/314 [==============================] - 1s 2ms/step - loss: 4.7754e-04 - val_loss: 4.4006e-04
Epoch 5/20
314/314 [==============================] - 1s 3ms/step - loss: 4.1329e-04 - val_loss: 3.8284e-04
Epoch 6/20
314/314 [==============================] - 1s 3ms/step - loss: 3.7264e-04 - val_loss: 3.3549e-04
Epoch 7/20
314/314 [==============================] - 1s 3ms/step - loss: 3.3355e-04 - val_loss: 4.3245e-04
Epoch 8/20
314/314 [==============================] - 1s 2ms/step - loss: 3.0845e-04 - val_loss: 3.4789e-04
Epoch 9/20
314/314 [==============================] - 1s 3ms/step - loss: 2.8654e-04 - val_loss: 2.6336e-04
Epoch 10/20
314/314 [===================

In [55]:
autoencoder.summary()

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
dense_37 (Dense)             (None, 200)               6200      
_________________________________________________________________
dense_38 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_39 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_40 (Dense)             (None, 200)               20200     
_________________________________________________________________
dense_41 (Dense)             (None, 30)                6030      
Total params: 62,630
Trainable params: 62,630
Non-trainable params: 0
_________________________________________________

In [67]:
# Building a computation graph to get the hidden representations of X
hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])
hidden_representation.add(autoencoder.layers[2])

In [68]:
# Obtaining the hidden representations of X_train
rep_X_train = hidden_representation.predict(X_train_nn)

In [69]:
# Logistic Regression - mapping NN Training output to y
logreg = LogisticRegression(solver='lbfgs')
logreg.fit(rep_X_train, y_train_nn)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [70]:
# Now moving on to the Dev Set
rep_X_dev = hidden_representation.predict(X_dev_nn)
nn_y_dev_preds = logreg.predict(rep_X_dev)
np.unique(nn_y_dev_preds)

array([0, 1], dtype=int64)

In [71]:
# Evaluating the Dev Set
evaluate_model(y_dev_nn, nn_y_dev_preds)

Recall score:
 0.5882352941176471
Precision score:
 0.9230769230769231
Confusion matrix:

               Normal (predicted)  Fraud (predicted)
Normal (true)               19995                  5
Fraud (true)                   42                 60


In [80]:
# Random Forest Classifier - training on hidden representations of X
rf = RandomForestClassifier(random_state=42)
rf.fit(rep_X_train, y_train_nn)

C:\Users\ZH834BT\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [82]:
# Evaluating on the Dev Set
nn_y_dev_preds_rf = rf.predict(rep_X_dev)
evaluate_model(y_dev_nn, nn_y_dev_preds_rf)

Recall score:
 0.803921568627451
Precision score:
 0.9425287356321839
Confusion matrix:

               Normal (predicted)  Fraud (predicted)
Normal (true)               19995                  5
Fraud (true)                   20                 82
